In [30]:
from langchain.document_loaders import PyPDFLoader

from langchain.text_splitter import CharacterTextSplitter
from langchain.llms import Ollama
from langchain.embeddings.ollama import OllamaEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI
import textwrap
import time


In [25]:
model_name ='phi3'

In [4]:
loader = PyPDFLoader("docs\\gym supplements\\1. Analysis of Actual Fitness Supplement.pdf") 
docs = loader.load()
len(docs)

15

In [5]:
def format_text(docs):
    return '\n\n'.join([x.page_content for x in docs])

context = format_text(docs)
len(context)

58287

In [6]:
print(context[0:100])

Citation: Espeño, P .R.; Ong, A.K.S.;
German, J.D.; Gumasing, M.J.J.; Casas,
E.S. Analysis of Actual


In [15]:
text_splitter = CharacterTextSplitter(
    separator="\n", chunk_size=512, chunk_overlap=100, length_function=len
)
chunks = text_splitter.split_text(context)
len(chunks)

152

In [ ]:
embeddings = OllamaEmbeddings(
    model=model_name,
)
knowledge_base = FAISS.from_texts(chunks, embeddings)

In [26]:
user_question = "talk about Supplementary drinks"
docs = knowledge_base.similarity_search(user_question)

base_url = "http://localhost:11434"

llm = Ollama(base_url=base_url,
                 model=model_name,
                 temperature=0.1,
                 num_predict=256
                 )
llm

Ollama(model='phi3', num_predict=256, temperature=0.1)

In [27]:
chain = load_qa_chain(llm, chain_type="stuff")
response = chain.run(input_documents=docs, question=user_question)

In [31]:
textwrap.wrap(response)

['The provided context does not directly mention supplementary drinks or',
 'provide specific information regarding them within the studies cited',
 '(Bhandari, Pinho et al., and Lino et al.). The focus of these pieces',
 'seems to be on vitamin D supplementation intention among farmers using',
 'pesticides as a safety behavior. Additionally, there is mention of',
 'healthy eating behaviors influenced by perceived barriers in the',
 'context provided by Pinho et al.  However, if we were to infer based',
 'on similar frameworks like TPB (Theory of Planned Behavior), HBM',
 '(Health Belief Model), and TEMPA (Towards Exercise Motivation through',
 'Physical Activity) that are used in the studies mentioned:   1. In a',
 'hypothetical study about supplementary drinks, perceived barriers',
 "could negatively affect individuals' intentions to consume these",
 'beverages regularly as they might consider them unnecessary or',
 'unhealthy if not properly educated on their benefits and appropriat